# Import packages and data 'CarPrice_Assignment.csv' from Kaggle
### https://www.kaggle.com/datasets/hellbuoy/car-price-prediction

In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import f,norm,sem
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.weightstats as sms
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from timeit import default_timer as timer
from sklearn.inspection import permutation_importance

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('CarPrice_Assignment.csv')
#Set view columns to max to avoid truncated data
pd.set_option('display.max_columns', None)

dictionary = pd.read_excel('Data Dictionary - carprices.xlsx')
dictionary = dictionary[['Unnamed: 7','Unnamed: 11']]

In [ ]:
df.head()

In [ ]:
dictionary = dictionary.dropna()
col_names = ['Variable', 'Description']
dictionary.columns = col_names
dictionary.head()

# Check Data and Clean

In [ ]:
#Check for missing values
df.isnull().sum()

In [ ]:
#Make new df for cleaned data, keep a copy of the original df if needed later
dfc = df.copy()

In [ ]:
#Car_ID is unique for each car so it is dropped
dfc = dfc.drop('car_ID',axis=1)

In [ ]:
dfc.dtypes

In [ ]:
#Symboling is not continous data but categorical, see data dictionary, change to object.
dfc['symboling'] = dfc['symboling'].astype(str)

#### Clean Car Names

In [ ]:
dfc['CarName'].unique()

In [ ]:
#There is few cars of the same model so I will replace the full car name with just the brand

#First make an array of all car brands in column

brands = ['alfa-romero','audi','bmw','chevrolet','dodge','honda','isuzu','jaguar','mazda','buick',
          'mercury','mitsubishi','nissan','peugeot','plymouth','porsche','renault','saab','subaru',
          'toyota','volkswagen','volvo']
list1 = df.CarName.unique()

#Use a loop to search for strings that contain a brand name and replace string with brand name, i.e. removing model name.

for i in range(len(brands)):
    brand = brands[i]
    l1 = [k for k in list1 if brand in k]
    dfc = dfc.replace(l1,brand)
    
#Replacing misspelled names manually

dfc = dfc.replace(['Nissan versa'], 'nissan')
dfc = dfc.replace(['vokswagen rabbit'], 'volkswagen')
dfc = dfc.replace(['porcshce panamera'], 'porsche')
dfc = dfc.replace([ 'maxda rx3', 'maxda glc deluxe'],'mazda')
dfc = dfc.replace(['toyouta tercel'],'toyota')
dfc = dfc.replace(['vw dasher','vw rabbit'],'volkswagen')

In [ ]:
dfc['CarName'].value_counts()

#### Checking other categorical variables

In [ ]:
dfc['enginelocation'].value_counts()

In [ ]:
dfc['symboling'].value_counts()

In [ ]:
dfc['doornumber'].value_counts()

In [ ]:
dfc['enginetype'].value_counts()

In [ ]:
dfc['aspiration'].value_counts()

In [ ]:
dfc['cylindernumber'].value_counts()

In [ ]:
#The vast majority of cars are front engine, this will not help in regression so is dropped
dfc = dfc.drop('enginelocation',axis=1)

In [ ]:
#Making a copy of the cleaned data.
df_cleaned = dfc.copy()

In [ ]:
np.round(df.corr(),2)

In [ ]:
#Get all numeric data columns and use boxpplots to check for skewed data.
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numericdf = df.select_dtypes(include=numerics)

for i in range(len(numericdf.columns)):
    plt.boxplot(numericdf[numericdf.columns[i]])
    plt.title(numericdf.columns[i])
    plt.show()

#### Price looks skewed, may use the log 10 function in MLR model.

# Data Exploration

## Is there a difference in mpg based on fuel types ?

In [ ]:
#Make a new column for average mpg, take the mean average of city and highway mpg combined.
dfc['averagempg'] = (dfc['citympg'] + dfc['highwaympg'])/2

In [ ]:
#Visualise the data on a boxplot

#Split the data into gas and diesel
gasmpg = dfc[dfc['fueltype']=='gas']['averagempg']
dieselmpg = dfc[dfc['fueltype']=='diesel']['averagempg']

#Plotting on boxplot
plt.boxplot([gasmpg.values,dieselmpg.values])
plt.xticks([1,2],['gas','diesel'])
plt.ylabel('Average Miles per Gallon')
plt.title('Mpg by Fueltype')
plt.show()

print('Gas mean average mpg: ',gasmpg.mean(), '\nfrom sample size: ', len(gasmpg))
print('\nDiesel mean average mpg: ',dieselmpg.mean(), '\nfrom sample size: ', len(dieselmpg))

In [ ]:
dfc['averagempg'].idxmax()

In [ ]:
df.iloc[30,:]

In [ ]:
#Fisher's F test
var_gas = np.var(gasmpg,ddof=1)
var_diesel = np.var(dieselmpg,ddof=1)
print(var_gas)
print(var_diesel)

In [ ]:
f_ratio = var_diesel/var_gas
print(f_ratio)

In [ ]:
#Get p value for difference in variances
dfn = len(dieselmpg)-1
dfd = len(gasmpg)-1
alpha = 0.05
p_one_tailed = f.sf(f_ratio, dfn, dfd)
p = p_one_tailed * 2
print('pvalue is: ',p)
if p < alpha:
    print('Reject the null hypothesis that variances of the samples are equal')
else:
    print('Null hypothesis cannot be rejected')

In [ ]:
#Use a pooled standard deviation to perform t-test
stats.ttest_ind(a = gasmpg, b = dieselmpg, equal_var=True)

In [ ]:
cm = sms.CompareMeans(sms.DescrStatsW(gasmpg), sms.DescrStatsW(dieselmpg))
print(cm.tconfint_diff(usevar='pooled'))

#### At 0.05 significance diesel cars get between 2.01 and 8.05 more mpg than gas. However, a gas Honda Civic has the best average
#### mpg in the sample.

#### Visualise distribution of drivewheels by carbody type

In [ ]:
#Define a funciton to print the value counts of drivewheels by carbody type
def get(name):
    a = dfc[dfc['carbody']==name]['drivewheel']
    print(name,' have the following drivewheels\n',a.value_counts(), '\n \n')

#Use a loop to put each carbody type through the above function

for i in range(len(dfc['carbody'].unique())):
    get(dfc['carbody'].unique()[i])

In [ ]:
#Organise results into categories and frequency
body = ("Convertible", "Hatchback", "Sedan","Wagon","Hardtop")
count = {
    'front wheel drive': (1,49,57,12,1),
    'rear wheel drive': (5,19,36,9,7),
    '4 wheel drive': (0,2,3,4,0),
}

x = np.arange(len(body))
width = 0.25
multiplier = 0

#Plot grouped bar chart
fig, ax = plt.subplots(figsize =(8,8))

for attribute, measurement in count.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    multiplier += 1



ax.set_ylabel('Number of cars')
ax.set_title('Drivewheel by bodytype')
ax.set_xticks(x + width, body)
ax.legend(loc='upper left')
ax.set_ylim(0, 65)
ax.set_xticklabels(["0","Convertible", "Hatchback", "Sedan","Wagon","Hardtop"])

plt.show()

#### Hatchbacks and Sedans are predominantely front wheel drive. They are also the most common car body type in the sample.
#### Convertibles and Hardtops are predominantely rear wheel drive, although the samples are small (n < 10)
#### Wagons have a mix of all 3 drivewheels.

## What are the best and worst performing cars in the data ?

In [ ]:
#Horsepower is a good metric to use to determine performance.
#However, power to weight ratio is a better metric for sports car perfomance as a car that has a high
#horsepower but is heavy will be slow.

#Make new column with power to weight ratio (horsepower / curbweight) multiplied by 2000 to convert lbs to US ton.
#Unit horsepower per US ton.


dfc['bhp/weight'] = (dfc['horsepower']/dfc['curbweight'])*2000

In [ ]:
#Find the largest values in new column created
dfc['bhp/weight'].nlargest(n=5)

In [ ]:
#Make a list of the index of the cars with best power to weight ratio
best = dfc['bhp/weight'].nlargest(n=5).index.values

In [ ]:
print('Best performing cars\n')
#Use the original dataframe to get full name of the top 5 performing cars
for i in range(5):
    print(i+1, df['CarName'].iloc[best[i]], 'has a power to weight ratio of:' , round(dfc['bhp/weight'].iloc[best[i]],1), 'bhp per US ton. Price : ', df['price'].iloc[best[i]], ' USD\n')

In [ ]:
print('Worst performing cars\n')
#Repeat steps above for the worst cars
worst = dfc['bhp/weight'].nsmallest(n=5).index.values
for i in range(5):
    print(i+1, df['CarName'].iloc[worst[i]], 'has a power to weight ratio of:' , round(dfc['bhp/weight'].iloc[worst[i]],1), 'bhp per US ton. Price : ', df['price'].iloc[worst[i]], ' USD\n')

## Which brands have the best insurance ratings ? ('symboling', +3 is risky, -3 is very safe)

In [ ]:
#Create blank DataFrame for normalised symbol counts
dfs = pd.DataFrame({'Blank': [0]}, index =['-2','-1','0','1','2','3'])

In [ ]:
#Create a loop that adds a new column for each brand containing the normalised value counts
for i in range(len(brands)):
    symb = dfc[dfc['CarName']== brands[i]]['symboling']
    dfs[brands[i]] = symb.value_counts(normalize=True, dropna = False)

In [ ]:
#Drop blank
dfs1 = dfs.drop('Blank',axis=1)
#Transpose data frame and plot with color gradient. Red is risky, blue is safe.
dfs1 = dfs1.T
#Sort data by which brand has the highest percetage of good symbols.
dfs1 = dfs1.sort_values(by=['-2','-1','0','1'],ascending=False)

#Convert normalise to percentage
dfs1 = dfs1 * 100

In [ ]:
dfs1.plot(
    kind = 'bar',
    stacked = True,
    title = 'Insurance symbols by car brand',
    mark_right = True,
    colormap='bwr',
    rot=65,
    figsize = (10,10),
    ylabel= 'Percentage of cars in sample',
    xlabel= 'Brand')

#### Volvo has the best insurance ratings while Saab has the worst in our given sample

## Do different drivewheels tend to have a difference in horsepower ?

In [ ]:
dfcyl = df_cleaned.copy()

In [ ]:
#split data
rwd = dfcyl[dfcyl['drivewheel']=='rwd']['horsepower']
fwd = dfcyl[dfcyl['drivewheel']=='fwd']['horsepower']
fourwd = dfcyl[dfcyl['drivewheel']=='4wd']['horsepower']

In [ ]:
#Plotting on boxplot
plt.boxplot([rwd.values,fwd.values,fourwd.values])
plt.xticks([1,2,3],['rear wheel drive','front wheel drive','four wheel drive'])
plt.ylabel('horsepower')
plt.title('horsepower by drivewheel')
plt.show()

#### Rear wheel drive may have slightly more horsepower. Use an ANOVA test to find if the means of the grouped data are different.

In [ ]:
f_oneway(rwd, fwd, fourwd)

#### There is statistical difference between the means of at least two of the groups.

In [ ]:
#perform Tukey post-hoc test.
print(pairwise_tukeyhsd(endog=dfcyl['horsepower'],
                        groups=dfcyl['drivewheel'],
                        alpha=0.05))

#### Rwd has 38.4 and 47.7 hp more than 4wd and fwd respectively at 0.05 significance.

# Make MLR model

In [ ]:
#Include all numeric and categorical variables for MLR model.
#New df for the model, dfm.

dfm = df_cleaned.copy()

In [ ]:
dfm = pd.get_dummies(dfm, drop_first=True)


In [ ]:
dfm.columns

In [ ]:
#Use scatter graphs to find relationship between variables and log10 of the price.

dfm = pd.get_dummies(dfm, drop_first = True)

for i in range(13):
    plt.scatter(dfm[dfm.columns[i]], np.log10(dfm['price']))
    plt.title(dfm.columns[i])
    plt.show()

In [ ]:
#Looks like a strong correlation between carwidth, curbweight, horsepower, enginesize and city mpg with log 10 of the price.
#Make a model with these variables
X = dfm[['carwidth','curbweight','horsepower','enginesize','citympg']]
X = sm.add_constant(X)

dfm['log10 price'] = np.log10(dfm['price'])
y = dfm['log10 price']


In [ ]:
#Define a function to make a MLR model an print a summary of results. 


regr = linear_model.LinearRegression()

def get_stats():
    regr.fit(X, y)
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X) 

    print_model = model.summary()
    print(print_model)

In [ ]:
get_stats()

#### Remove variable with highest p value, re-run model and repeat until all variables have p less than 0.05.

In [ ]:
X = dfm[['carwidth','curbweight','horsepower','enginesize']]
X = sm.add_constant(X)
get_stats()

In [ ]:
X = dfm[['carwidth','curbweight','horsepower']]
X = sm.add_constant(X)
get_stats()

In [ ]:
#drop car width to reduce multicollinearity
X = X.drop('carwidth', axis = 1)
get_stats()

In [ ]:
fig = plt.figure(figsize = (12, 8))

model = sm.OLS(y, X).fit()

fig = plt.figure(figsize = (12, 8))
fig = sm.graphics.plot_regress_exog(model, 'curbweight', fig = fig)
plt.show()

In [ ]:
fig = plt.figure(figsize = (12, 8))
fig = sm.graphics.plot_regress_exog(model, 'horsepower', fig = fig)
plt.show()

#### Horsepower is right skewed, try log10 for horsepower.

In [ ]:
X['log10 horsepower'] = np.log10(X['horsepower'])
X = X.drop('horsepower', axis = 1)

In [ ]:
get_stats()

In [ ]:
fig = plt.figure(figsize = (12, 8))

model = sm.OLS(y, X).fit()

fig = sm.graphics.plot_regress_exog(model, 'log10 horsepower', fig = fig)
plt.title('Regrssion Plots for log10 Horsepower')
plt.show()

In [ ]:
results = model
# Get different Variables for diagnostic
residuals = results.resid
fitted_value = results.fittedvalues
stand_resids = results.resid_pearson
influence = results.get_influence()
leverage = influence.hat_matrix_diag
  
# PLot different diagnostic plots
plt.rcParams["figure.figsize"] = (20,15)
fig, ax = plt.subplots(nrows=2, ncols=2)
  
plt.style.use('seaborn')
  
# Residual vs Fitted Plot
sns.scatterplot(x=fitted_value, y=residuals, ax=ax[0, 0])
ax[0, 0].axhline(y=0, color='grey', linestyle='dashed')
ax[0, 0].set_xlabel('Fitted Values')
ax[0, 0].set_ylabel('Residuals')
ax[0, 0].set_title('Residuals vs Fitted Fitted')
  
# Normal Q-Q plot
sm.qqplot(residuals, fit=True, line='45',ax=ax[0, 1], c='#4C72B0')
ax[0, 1].set_title('Normal Q-Q')
  
# Scale-Location Plot
sns.scatterplot(x=fitted_value, y=residuals, ax=ax[1, 0])
ax[1, 0].axhline(y=0, color='grey', linestyle='dashed')
ax[1, 0].set_xlabel('Fitted values')
ax[1, 0].set_ylabel('Sqrt(standardized residuals)')
ax[1, 0].set_title('Scale-Location Plot')
  
# Residual vs Leverage Plot
sns.scatterplot(x=leverage, y=stand_resids, ax=ax[1, 1])
ax[1, 1].axhline(y=0, color='grey', linestyle='dashed')
ax[1, 1].set_xlabel('Leverage')
ax[1, 1].set_ylabel('Sqrt(standardized residuals)')
ax[1, 1].set_title('Residuals vs Leverage Plot')
  
  
plt.tight_layout()
plt.show()
  
# PLot Cook's distance plot
sm.graphics.influence_plot(results, criterion="cooks")

In [ ]:
df.iloc[126:129,:]

In [ ]:
dfm.columns

In [ ]:
X = dfm[['curbweight','horsepower',
       'symboling_-2', 'symboling_0', 'symboling_1', 'symboling_2',
       'symboling_3', 'CarName_audi', 'CarName_bmw',
       'CarName_buick', 'CarName_chevrolet', 'CarName_dodge', 'CarName_honda',
       'CarName_isuzu', 'CarName_jaguar', 'CarName_mazda', 'CarName_mercury',
       'CarName_mitsubishi', 'CarName_nissan', 'CarName_peugeot',
       'CarName_plymouth', 'CarName_porsche', 'CarName_renault',
       'CarName_saab', 'CarName_subaru', 'CarName_toyota',
       'CarName_volkswagen', 'CarName_volvo',
       'fueltype_gas', 'aspiration_turbo',
       'doornumber_two', 'carbody_hardtop',
       'carbody_hatchback', 'carbody_sedan', 'carbody_wagon',
       'drivewheel_fwd', 'drivewheel_rwd',
       'enginetype_dohcv', 'enginetype_l', 'enginetype_ohc', 'enginetype_ohcf',
       'enginetype_ohcv', 'enginetype_rotor',
       'cylindernumber_five', 'cylindernumber_four', 'cylindernumber_six',
       'cylindernumber_three', 'cylindernumber_twelve', 'cylindernumber_two',
         'fuelsystem_2bbl', 'fuelsystem_4bbl',
       'fuelsystem_idi', 'fuelsystem_mfi', 'fuelsystem_mpfi',
       'fuelsystem_spdi', 'fuelsystem_spfi']]
X = sm.add_constant(X)
X['log10 horsepower'] = np.log10(X['horsepower'])
X = X.drop('horsepower', axis = 1)

In [ ]:
get_stats()

In [ ]:
X = dfm[['curbweight','horsepower','symboling_-2','CarName_bmw','CarName_mitsubishi','CarName_peugeot',
         'CarName_plymouth','CarName_subaru','CarName_toyota','fueltype_gas','carbody_hatchback',
         'carbody_wagon','enginetype_ohcf','cylindernumber_five','fuelsystem_idi','fuelsystem_mpfi']]
X = sm.add_constant(X)
X['log10 horsepower'] = np.log10(X['horsepower'])
X = X.drop('horsepower', axis = 1)

In [ ]:
get_stats()

In [ ]:
X = dfm[['curbweight','horsepower','symboling_-2','CarName_bmw','CarName_mitsubishi','CarName_peugeot',
         'CarName_plymouth','CarName_subaru','CarName_toyota','fueltype_gas','carbody_hatchback',
         'carbody_wagon','enginetype_ohcf','fuelsystem_idi','fuelsystem_mpfi']]
X = sm.add_constant(X)
X['log10 horsepower'] = np.log10(X['horsepower'])
X = X.drop('horsepower', axis = 1)
get_stats()

In [ ]:
X = X.drop('symboling_-2',axis = 1)
get_stats()

In [ ]:
X = X.drop('carbody_hatchback',axis = 1)
get_stats()

In [ ]:
X = X.drop('CarName_plymouth',axis = 1)
get_stats()

In [ ]:
XMLR = X

model = sm.OLS(y, X).fit()
results = model
# Get different Variables for diagnostic
residuals = results.resid
fitted_value = results.fittedvalues
stand_resids = results.resid_pearson
influence = results.get_influence()
leverage = influence.hat_matrix_diag
  
# PLot different diagnostic plots
plt.rcParams["figure.figsize"] = (20,15)
fig, ax = plt.subplots(nrows=2, ncols=2)
  
plt.style.use('seaborn')
  
# Residual vs Fitted Plot
sns.scatterplot(x=fitted_value, y=residuals, ax=ax[0, 0])
ax[0, 0].axhline(y=0, color='grey', linestyle='dashed')
ax[0, 0].set_xlabel('Fitted Values')
ax[0, 0].set_ylabel('Residuals')
ax[0, 0].set_title('Residuals vs Fitted Fitted')
  
# Normal Q-Q plot
sm.qqplot(residuals, fit=True, line='45',ax=ax[0, 1], c='#4C72B0')
ax[0, 1].set_title('Normal Q-Q')
  
# Scale-Location Plot
sns.scatterplot(x=fitted_value, y=residuals, ax=ax[1, 0])
ax[1, 0].axhline(y=0, color='grey', linestyle='dashed')
ax[1, 0].set_xlabel('Fitted values')
ax[1, 0].set_ylabel('Sqrt(standardized residuals)')
ax[1, 0].set_title('Scale-Location Plot')
  
# Residual vs Leverage Plot
sns.scatterplot(x=leverage, y=stand_resids, ax=ax[1, 1])
ax[1, 1].axhline(y=0, color='grey', linestyle='dashed')
ax[1, 1].set_xlabel('Leverage')
ax[1, 1].set_ylabel('Sqrt(standardized residuals)')
ax[1, 1].set_title('Residuals vs Leverage Plot')
  
  
plt.tight_layout()
plt.show()

## This is our best model, adjusted R-squared of 0.924 and an AIC of -557.8

#### Create test and train datasets to find the accuracy of the above MLR model.

In [ ]:
#Split data.

X = XMLR

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 30)
regr = linear_model.LinearRegression()

start1 = timer()

regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

end1 = timer()

MLR = round(metrics.r2_score(y_test, y_pred), 2)
print(regr)
print('RMSE = ', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),4))
print("R-squared score =", round(metrics.r2_score(y_test, y_pred), 2))
print("Explain variance score =", round(metrics.explained_variance_score(y_test, y_pred), 2))

#Use k-fold cross validation to assess the model
    

scores = cross_val_score(regr, X = X_train, y = y_train, cv = 10, n_jobs = 1)
print('K-fold cross validation score accuracy: ', np.round(np.mean(scores),3), '+/-', np.round(np.std(scores),3))

# Random Forest Regrssion

In [ ]:
#Include all variables in the first random forest regression.
regr = RandomForestRegressor(n_estimators=100)
X = dfm.drop(['price','log10 price'],axis=1)
y = dfm['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 30)


regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)


RFR = round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),4)
print(regr)
print('RMSE = ', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),4))
print("R-squared score =", round(metrics.r2_score(y_test, y_pred), 2))
print("Explain variance score =", round(metrics.explained_variance_score(y_test, y_pred), 2))

#Use k-fold cross validation to assess the model
    

scores = cross_val_score(regr, X = X_train, y = y_train, cv = 10, n_jobs = 1)
print('K-fold cross validation score accuracy: ', np.round(np.mean(scores),3), '+/-', np.round(np.std(scores),3))

In [ ]:
#Find importance of features.
feature_imp = pd.Series(regr.feature_importances_,X_train.columns).sort_values(ascending=False)

sorted_idx = regr.feature_importances_.argsort()
plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(12,12),dpi=80)
plt.xlabel("Permutation Importance")
plt.barh(X_test.columns[sorted_idx], regr.feature_importances_[sorted_idx])
plt.title('Importance in Random Forest Regression')
plt.show()

In [ ]:
#The following variables give the best accuracy 
X = dfm[['curbweight','enginesize','highwaympg','horsepower','carwidth','citympg','wheelbase']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 30)

start2 = timer()

regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

end2 = timer()

RFR = round(metrics.r2_score(y_test, y_pred), 2)
print(regr)
print('RMSE = ', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),4))
print("R-squared score =", round(metrics.r2_score(y_test, y_pred), 2))
print("Explain variance score =", round(metrics.explained_variance_score(y_test, y_pred), 2))

#Use k-fold cross validation to assess the model
    

scores = cross_val_score(regr, X = X_train, y = y_train, cv = 10, n_jobs = 1)
print('K-fold cross validation score accuracy: ', np.round(np.mean(scores),3), '+/-', np.round(np.std(scores),3))

# PCA with MLR

In [ ]:
dfp = df_cleaned.copy()

In [ ]:
dfp = pd.get_dummies(dfp, drop_first = True)

In [ ]:
X = dfp.drop('price',axis = 1)
y = np.log10(dfp['price'])

In [ ]:
dfp_scaled = StandardScaler().fit_transform(X)
pca=PCA(n_components=30)
X_red = pca.fit_transform(dfp_scaled)

In [ ]:
print('Shape before PCA: ', dfp_scaled.shape)
print('Shape after PCA: ', X_red.shape)

In [ ]:
pca.explained_variance_

### Use the components with a value above 1

In [ ]:
np.cumsum(np.round(pca.explained_variance_ratio_, decimals = 4)*100)[0:21]

### 79.8 % is explained by 22 components, reduced from 67 originally.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 30)

In [ ]:
X_red_train = pca.fit_transform(StandardScaler().fit_transform(X_train))
X_red_test = pca.transform(StandardScaler().fit_transform(X_test))[:,0:21]
lm = LinearRegression()

start3 = timer()

pcr = lm.fit(X_red_train[:,0:21], y_train)
y_pred = pcr.predict(X_red_test)

end3 = timer()

In [ ]:
PC = round(metrics.r2_score(y_test, y_pred), 2)
print('RMSE: ', np.round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),4))
print("R-squared score =", round(metrics.r2_score(y_test, y_pred), 2))

# Evaluate accuracy and efficiency of all three models

In [ ]:
print('Multiple Linear Regression R squared: ', MLR, 
      '\nRandom Forest Regression R squared: ', RFR, 
      '\nPrincipal Component Analysis R squared:', PC)


In [ ]:
print('Time for Multiple Linear Regression: ', np.round(end1 - start1, 4), 's')
print('Time for Random Forest: ', np.round(end2 - start2, 4), 's')
print('Time for Principal Component Analysis: ', np.round(end3 - start3, 4), 's')

In [ ]:
data = {'Method': ['Multiple Linear Regression','Random Forest Regression','Principal Component Analysis'],
        'R Squared score on test data': [MLR, RFR, PC],
        'Time (s)': [np.round(end1 - start1, 4), np.round(end2 - start2, 4), np.round(end3 - start3, 4)]}
df_final = pd.DataFrame(data=data)
df_final

# Final Remarks

### All three methods have similar accuracies after being tested on the same test data.
### Each method provides a good prediction for the price of a car.
### Random Forest is the most accurate but slow.
### PCA is a little less accurate but is 100 fold faster than Random Forest which may be important for a larger sample.
### MLR has good accuracy in a reasonable time. It is the best model to use.